In [23]:
from scripts.data_util.sqlConnector import online_sql_connector , update_connector
update_connector(path = './data/DB_data/DB_3rdPartySQL.h5' , connector = online_sql_connector() , trace = 1)


haitong/hf_factors
since 20130101 to 20231227, total 43 periods(Q)


Traceback (most recent call last):
  File "/home/mengkjin/Workspace/learndl/scripts/data_util/sqlConnector.py", line 173, in default_query
    df = pd.read_sql_query(query , self.connections[src])
  File "/home/mengkjin/.local/lib/python3.10/site-packages/pandas/io/sql.py", line 486, in read_sql_query
    return pandas_sql.read_query(
  File "/home/mengkjin/.local/lib/python3.10/site-packages/pandas/io/sql.py", line 1776, in read_query
    result = self.execute(sql, params)
  File "/home/mengkjin/.local/lib/python3.10/site-packages/pandas/io/sql.py", line 1599, in execute
    return self.con.exec_driver_sql(sql, *args)
  File "/home/mengkjin/.local/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1778, in exec_driver_sql
    ret = self._execute_context(
  File "/home/mengkjin/.local/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1848, in _execute_context
    return self._exec_single_context(
  File "/home/mengkjin/.local/lib/python3.10/site-packages/sqlalche

In [22]:
from sqlalchemy import create_engine , exc , text
from datetime import date , datetime
import traceback
import pandas as pd
import numpy as np
import os , time

connect_dict = {
    'haitong':{
        'dialect'  : 'mssql+pyodbc' ,
        'driver'   : 'FreeTDS' ,
        'username' : 'JSJJDataReader' ,
        'password' : 'JSJJDataReader' ,
        'host'     : 'rm-uf6777pp2098v0um6hm.sqlserver.rds.aliyuncs.com' ,
        'port'     : 3433 ,
        'database' : 'daily_factor_db' ,
    },
    'dongfang':{
        'dialect'  : 'mysql+pymysql' ,
        'username' : 'hfq_jiashi' ,
        'password' : 'LTBi94we' ,
        'host'     : '139.196.77.199' ,
        'port'     : 81 ,
        'database' : 'model' ,
    },
    'kaiyuan':{
        'dialect'  : 'postgresql' ,
        'username' : 'harvest_user' ,
        'password' : 'harvest' ,
        'host'     : '1.15.124.26' ,
        'port'     : 5432 ,
        'database' : 'kyfactor' ,
    },
    'guojin':{
        'dialect'  : 'mysql+pymysql' ,
        'username' : 'jsfund' ,
        'password' : 'Gjquant_js!' ,
        'host'     : 'quantstudio.tpddns.cn' ,
        'port'     : 3306 ,
        'database' : 'gjquant' ,
    },
}

data_start_dt = {
    'haitong':{'hf_factors': 20130101,
               'dl_factors': 20161230},
    'dongfang':{'hfq_chars'     : 20050101 ,
                'l2_chars'      : 20130930 ,
                'ms_chars'      : 20050101 ,
                'order_flow'    : 20130930 ,
                'gp'            : 20170101 ,
                'tra'           : 20200101} ,
    'kaiyuan':{
        'positive': 20140130,
        'negative': 20140130},
    'guojin':{}
}
query_params = {
    'haitong':{'hf_factors':{}, 'dl_factors':{},},
    'dongfang':{'hfq_chars' :{'date_col':'tradingdate','date_fmt':'%Y%m%d'},
                'l2_chars'  :{'date_col':'trade_date' ,'date_fmt':'%Y%m%d'},
                'ms_chars'  :{'date_col':'trade_date' ,'date_fmt':'%Y%m%d'},
                'order_flow':{'date_col':'trade_date' ,'date_fmt':'%Y%m%d'},
                'gp'        :{'date_col':'tradingdate','date_fmt':'%Y-%m-%d'},
                'tra'       :{'date_col':'tradingdate','date_fmt':'%Y-%m-%d'},
                'scores_v0' :{'date_col':'tradingdate','date_fmt':'%Y-%m-%d'},
                } ,
    'kaiyuan':{
        'positive':{'factors' : 
                    ['active_trading','apm','opt_synergy_effect','large_trader_ret_error',
                     'offense_defense','high_freq_shareholder','pe_change',],},
        'negative':{'factors' : 
                    ['smart_money','ideal_vol','ideal_reverse','herd_effect','small_trader_ret_error',],},
        # +1 : 'traction_f' , 'traction_ns' , 'traction_si', , 'long_momentum2' , 'merge_sue', 'consensus_adjustment',
    },
    'guojin':{},
}

# class online_sql_connector
class online_sql_connector():
    def __init__(self):
        self.connect_dict = connect_dict
        self.engines = dict()
        self.connections = dict()
        self.data_start_dt = data_start_dt
        self.query_params = query_params
        [self.create_engines(src) for src in self.connect_dict.keys()]

    def create_engines(self , srcs):
        if not isinstance(srcs , (list,tuple)): srcs = [srcs]
        for src in srcs:
            self.engines[src] = self._single_engine(self.connect_dict[src])
            
    def create_connections(self , srcs):
        if not isinstance(srcs , (list,tuple)): srcs = [srcs]
        for src in srcs:
            if self.connections.get(src) is not None:
                self.connections.get(src).close()
            self.connections[src] = self.engines[src].connect()

    def close_all(self):
        [connection.close() for _,connection in self.connections.items()]
        
    def _single_engine(self , src_dict):
        connect_url = '{dialect}://{username}:{password}@{host}:{port}/{database}'.format(
            dialect  = src_dict['dialect'],
            username = src_dict['username'],
            password = src_dict['password'],
            host     = src_dict['host'],
            port     = src_dict['port'],
            database = src_dict['database'],
        )
        
        if src_dict.get('driver'): connect_url += f'?driver={src_dict.get("driver")}'
        return create_engine(connect_url)
    
    def _single_default_query(self , src , query_type , start_dt , end_dt):
        assert query_type in self.query_params[src].keys()
        if src == 'haitong':
            if query_type == 'hf_factors':
                query = '''
                select *  
                from daily_factor_db.dbo.JSJJHFFactors t 
                where t.trade_dt between \'{}\' and \'{}\'
                '''.format(start_dt , end_dt)
            else:
                query = '''
                select s_info_windcode , trade_dt , f_value , model 
                from daily_factor_db.dbo.JSJJDeepLearnFactorsV2 t 
                where t.trade_dt between \'{}\' and \'{}\'
                '''.format(start_dt , end_dt)
        elif src == 'dongfang':
            date_col = self.query_params[src][query_type]['date_col']
            date_fmt = self.query_params[src][query_type]['date_fmt']
            query = '''
            select *  
            from {} t 
            where t.{} between \'{}\' and \'{}\'
            '''.format(query_type , date_col , 
                        datetime.strptime(str(start_dt) , '%Y%m%d').strftime(date_fmt) ,
                        datetime.strptime(str(end_dt) , '%Y%m%d').strftime(date_fmt))
        elif src == 'kaiyuan':
            query = {v:f'select * from public.{v} where date >= \'{start_dt}\' and date <= \'{end_dt}\''
                    for v in self.query_params[src][query_type]['factors']}
        elif src == 'guojin':
            query = 'select * from gjquant.factordescription'
        return query
    
    def query_start_dt(self , src , query_type):
        assert query_type in self.query_params[src].keys()
        if src == 'haitong':
            if query_type == 'hf_factors':
                query = 'select min(trade_dt) from daily_factor_db.dbo.JSJJHFFactors'
            else:
                query = 'select min(trade_dt) daily_factor_db.dbo.JSJJDeepLearnFactorsV2'
        elif src == 'dongfang':
            date_col = self.query_params[src][query_type]['date_col']
            query = 'select min({}) from {}'.format(date_col , query_type)
        elif src == 'kaiyuan':
            query = 'select min(date) from public.smart_money'
        elif src == 'guojin':
            return 99991231
        return pd.read_sql_query(query , self.connections[src])

    def default_query(self , src , query_type , start_dt = 20231101, end_dt = 20231103):
        query = self._single_default_query(src , query_type , start_dt , end_dt)
        if src not in self.connections.keys(): self.create_connections(src)
        try:
            if src == 'kaiyuan':
                df = {k:pd.read_sql_query(q , self.connections[src]) for k,q in query.items()}
            else:
                df = pd.read_sql_query(query , self.connections[src])
        except exc.ResourceClosedError:
            print(f'{src} Connection is closed, re-connect')
            self.create_connections(src)
            if src == 'kaiyuan':
                pass
            else:
                df = pd.read_sql_query(query , self.connections[src])
        except:
            raise Exception
        df = self.df_process(df , src , query_type)
        return df
    
    def df_process(self , df , src , query_type):
        if src == 'haitong':
            df.columns = map(str.lower , df.columns)
            df = df.rename(columns={'s_info_windcode':'secid','trade_dt':'date'})
            df['date'] = df['date'].astype(int)
            df['secid'] = self._IDconvert(df['secid'])
            if query_type == 'hf_factors':
                df = df.reset_index(drop = True)
            else:
                df.loc[:,'model'] = 'haitong_dl_' + df.loc[:,'model'].astype(str)
                df = df.pivot_table('f_value',['date','secid'],'model').reset_index()
        elif src == 'dongfang':
            df.columns = map(str.lower , df.columns)
            df = df.rename(columns={'stockcode':'secid','ticker':'secid',
                                    'tradingdate':'date','trade_dt':'date' , 'trade_date':'date'})
            df['secid'] = df['secid'].str.replace('[-.a-zA-Z]','',regex=True).astype(int)
            df['date'] = df['date'].astype(str).str.replace('-','').astype(int)
        elif src == 'kaiyuan':
            df0 = pd.DataFrame(columns = ['date','code'])
            for k,subdf in df.items():
                subdf.rename(columns = {'factor':k})
                # print(subdf.iloc[:5])
                df0 = df0.merge(subdf.rename(columns = {'factor':k}),how='outer',on=['date','code'])
            df = df0.rename(columns={'code':'secid'})
            df['secid'] = df['secid'].str.replace('[-.a-zA-Z]','',regex=True).astype(int)
            df['date']  = df['date'].astype(int)
            df = df.set_index(['date','secid']).reset_index()
        elif src == 'guojin':
            pass
        return df.fillna(np.nan)
    
    def _IDconvert(self , x):
        if isinstance(x , (bytes)):
            return int(x.decode('utf-8').split('.')[0].split('!')[0])
        if isinstance(x , (str)):
            return int(x.split('.')[0].split('!')[0])
        else:
            return type(x)([self._IDconvert(xx) for xx in x])

    def download_since(self , dir , src , query_type , start_dt , end_dt , ask = True):
        os.makedirs(dir , exist_ok= True)
        file_name = os.path.join(dir , src , query_type , query_type + '_{}.pkl')
        os.makedirs(os.path.dirname(file_name) , exist_ok= True)
        end_dt = min(end_dt , int(date.today().strftime('%Y%m%d')))
        if end_dt < start_dt: return

        freq = 'Q'
        date_segs = self.date_seg(start_dt , end_dt , freq = freq)
        prompt = f'since {start_dt} to {end_dt}, total {len(date_segs)} periods({freq})'
        if ask:
            assert input(prompt + ', print "yes" to confirm!') == 'yes'
        else:
            print(prompt)

        for (s , e) in date_segs:
            data = self.default_query(src , query_type , s , e)
            data = data.sort_values(['date' , 'secid']).set_index('date')
            for d in data.index.unique():
                data.loc[d].to_pickle(file_name.format(d))
            print(f'{time.ctime()} : {src}:{query_type}:{s // 100}{" "*20}' , end='\r')
        print('\n')

    def date_seg(self , start_dt , end_dt , freq='Q'):
        dt_list = pd.date_range(str(start_dt) , str(end_dt) , freq=freq).strftime('%Y%m%d').astype(int)
        dt_starts = [start_dt , *self.numdate_offset(dt_list[:-1],1)]
        dt_ends = [*dt_list[:-1] , end_dt]
        return [(s,e) for s,e in zip(dt_starts , dt_ends)]
    
    def numdate_offset(self , date , offset):
        if offset == 0:
            return date
        else:
            if isinstance(date , (np.ndarray,pd.Index , pd.Series)):
                return (pd.DatetimeIndex(date.astype(str))+pd.DateOffset(offset)).strftime('%Y%m%d').astype(int).values
            else:
                return (pd.DatetimeIndex([str(date)])+pd.DateOffset(offset)).strftime('%Y%m%d').astype(int)[0]
                

connector = online_sql_connector()
DIR = 'test_data'
start_dt = 20230101
end_dt = 20230110
try:
    for src in connector.query_params.keys():
        for qtype in connector.query_params[src].keys():
            print('/'.join([src,qtype]))
            connector.download_since(DIR,src,qtype,start_dt,end_dt,ask=False)
except Exception as e:
    traceback.print_exc()


haitong/hf_factors
since 20230101 to 20230110, total 1 periods(Q)


Exception during reset or similar
Traceback (most recent call last):
  File "/home/mengkjin/.local/lib/python3.10/site-packages/pymysql/connections.py", line 779, in _read_bytes
    data = self._rfile.read(num_bytes)
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mengkjin/.local/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 988, in _finalize_fairy
    fairy._reset(
  File "/home/mengkjin/.local/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 1438, in _reset
    pool._dialect.do_rollback(self)
  File "/home/mengkjin/.local/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 692, in do_rollback
    dbapi_connection.rollback()
  File "/home/mengkjin/.local/lib/python3.10/site-packages/pymysql/connections.py", line 488, in roll

Wed Dec 27 23:07:26 2023 : haitong:hf_factors:202301                    

haitong/dl_factors
since 20230101 to 20230110, total 1 periods(Q)
Wed Dec 27 23:08:10 2023 : haitong:dl_factors:202301                    

dongfang/hfq_chars
since 20230101 to 20230110, total 1 periods(Q)
Wed Dec 27 23:08:17 2023 : dongfang:hfq_chars:202301                    

dongfang/l2_chars
since 20230101 to 20230110, total 1 periods(Q)
Wed Dec 27 23:08:41 2023 : dongfang:l2_chars:202301                    

dongfang/ms_chars
since 20230101 to 20230110, total 1 periods(Q)
Wed Dec 27 23:09:20 2023 : dongfang:ms_chars:202301                    

dongfang/order_flow
since 20230101 to 20230110, total 1 periods(Q)
Wed Dec 27 23:09:22 2023 : dongfang:order_flow:202301                    

dongfang/gp
since 20230101 to 20230110, total 1 periods(Q)
Wed Dec 27 23:09:23 2023 : dongfang:gp:202301                    

dongfang/tra
since 20230101 to 20230110, total 1 periods(Q)
Wed Dec 27 23:09:24 2023 : dongfang:tra:20

In [20]:
import pandas as pd
a = pd.DataFrame({'a':['ab-cc.c2','-ba.ccc4']})
a

,a
0,ab-cc.c2
1,-ba.ccc4


In [21]:
x = a['a'].str.replace('[-.a-zA-Z]','0',regex=True)
x

0    00000002
1    00000004
Name: a, dtype: object